# Generate Data for Chartbook

Brian Dew

@bd_econ

In [3]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [ ]:
#nipa_series_codes(retrieve_table('T10105'))

### GDP components

In [ ]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

s = ['DPCERX', 'A006RX', 'A019RX', 'A822RX']
s2 = ['DPCERC', 'A006RC', 'A019RC', 'A822RC']
df = nipa_df(retrieve_table('T10106')['Data'], s).sort_index()
df2 = nipa_df(retrieve_table('T10105')['Data'], s2).sort_index()

dfpop = df.div(pop, axis=0)
real_vals = df2.rename({i: i.replace('RC', 'RX') for i in s2}, axis=1).iloc[-1]
data = ((dfpop / dfpop.iloc[-1]) * (real_vals / pop.iloc[-1])).loc['1989':]

data.to_csv(data_dir / 'gdp_pc_comp.csv', index_label='date')

In [ ]:
pce = f'\${data.DPCERX.iloc[-1] * 1000:,.0f}'
inv = f'\${data.A006RX.iloc[-1] * 1000:,.0f}'
gov = f'\${data.A822RX.iloc[-1] * 1000:,.0f}'
exp = f'\${data.A019RX.iloc[-1] * 1000:,.0f}'
ltdt = dtxt(data.index[-1])['qtr1']
pce_diff = f'\${(data.DPCERX.iloc[-1] - data.DPCERX.iloc[0]) * 1000:,.0f}'

text = ('Much of the increase in real GDP per capita over the past '+
        '30 years comes from consumer spending. Domestic '+
        'consumer spending (see {\color{yellow!65!orange}\\textbf{---}}) '+
        f'is equivalent to {pce} per person in {ltdt}, a price-adjusted increase '+
        f'of {pce_diff} since 1989. Gross private domestic investment '+
        '(see {\color{blue!70!black}\\textbf{---}}) is equivalent to '+
        f'{inv} per person in {ltdt}, and government spending and '+
        'investment (see {\color{cyan!60!white}\\textbf{---}}) totals '+
        f'{gov} per person. To adjust the above to capture only domestic '+
        'production, net exports (see {\color{green!60!black}\\textbf{---}}), '+
        f'equivalent to {exp} per person in {ltdt}, '+
        'are included.')

write_txt(text_dir / 'gdp_pc_comp.txt', text)
text

### Labor Share

In [ ]:
s = ['A261RC', 'A4002C']

df = nipa_df(retrieve_table('T11000')['Data'], s).sort_index()
df['Share'] = (df['A4002C'] / df['A261RC']) * 100
data = df.loc['1989':, 'Share'].dropna()
data.to_csv(data_dir / 'laborshare.csv', 
            index_label='date', 
            float_format='%g')

node = end_node(data, 'blue!60!cyan')
write_txt(text_dir / 'laborshare_node.txt', node)

In [ ]:
s = series_info(data)
ltdate = dtxt(s['date_latest'])['qtr2']
one_yr = val_inc_pp(s['val_latest'] - s['val_year_ago'])
ltmin = s['val_latest'] - s['val_min']
ltmax = s['val_max'] - s['val_latest']
text = (f'As of {ltdate}, labor receives '+
        f'{s["val_latest"]:.1f} percent of gross domestic income, '+
        f'and the labor share {one_yr} over the past year. '+
        f'The labor share is {ltmin:.1f} percentage '+
        f'points above its 30-year low of {s["val_min"]:.1f} percent '+
        f'in {s["date_min_ft"]}, but {ltmax:.1f} percentage points '+
        f'below the 30-year high of {s["val_max"]:.1f} percent in '+
        f'{s["date_max_ft"]}. ')

write_txt(text_dir / 'laborshare.txt', text)
text

### GDP growth rate

In [ ]:
s = ['A191RL']
df = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
df.loc['1989':].to_csv(data_dir / 'gdp.csv', index_label='date')
date = dtxt(df.index[-1])['qtr1']

txt = f'{date}: {df["A191RL"].iloc[-1]}\%'
write_txt(data_dir / 'gdp.txt', txt)

In [ ]:
df = nipa_df(retrieve_table('T10502')['Data'], ['A191RL'])['A191RL']
ltdate = dtxt(df.index[-1])['qtr2']
prdate = dtxt(df.index[-2])['qtr2']
ltval = df.iloc[-1]
inc_dec = 'increased' if ltval >= 0 else 'decreased'
prval = df.iloc[-2]
inc_dec2 = 'increase' if prval >= 0 else 'decrease'
d = {}
for i in [-13, -41, -121]:
    d[i] = df.iloc[i:].values.mean().round(1)
    
text = (f'{inc_dec} at an annual rate of {ltval:.1f} percent during {ltdate}, '+
        f'compared to an {inc_dec2} of {prval:.1f} percent in {prdate}. This '+
        f'annualized rate of quarterly growth has averaged {d[-13]} percent over '+
        f'the past three years, {d[-41]} percent over the past 10 years, and '+
        f'{d[-121]} percent over the past 30 years.')    

write_txt(text_dir / 'gdp_gr.txt', text)

text

### GDP Intro Chart and Text

In [ ]:
df = nipa_df(retrieve_table('T70100')['Data'], ['A939RC', 'A939RX'])
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1]) * df['A939RC'].iloc[-1]
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
rgdp_cd = rgdp / rgdp.iloc[-1] * cd

ltdate = dtxt(df.index[-1])['qtr2']
gdp_val = int(rgdp_cd.iloc[-1].values[0] / 1000)
gdp_prv_val = int(rgdp_cd.iloc[0].values[0] / 1000)
gdppc_val = int(df.iloc[-1].loc['value'])
gdppc_prv_val = int(df.iloc[0].loc['value'])
qdate = dtxt(df.index[-1])['qtr1']

text = (f'\${gdp_val:,} billion in {ltdate}, compared to an '+
        f'inflation-adjusted equivalent of \${gdp_prv_val:,} '+
        'billion in the first quarter of 1989.')
write_txt(text_dir / 'gdp1.txt', text)
print(text, '\n\n')

text = ('The US population is growing by about sixth-tenths of a '+
        'percent per year. GDP per capita (see {\color{blue}\\textbf{---}} ), '+
        f'adjusted for inflation to {qdate} dollars, has increased to '+
        f'\${gdppc_val:,} in {qdate} from \${gdppc_prv_val:,} in 1989 Q1.')
write_txt(text_dir / 'gdp2.txt', text)
print(text, '\n\n')

date = df.index[-1].strftime('%Y-%m-%d')
ldate = dtxt(df.index[-1])['qtr4']

text = [r'\noindent \normalsize \textbf{GDP per capita}\\',
r'\footnotesize{\textit{in ' + qdate + r' dollars}}\\* ',
r'\noindent \begin{tikzpicture}',
r'    \begin{axis}[\dateaxisticks clip=false, height=4.2cm, width=5.8cm,',
r'            ytick={40000,50000,60000}, yticklabels={40, 50, \$60k},',
r'            xtick={{1989-01-01}, {2000-01-01}, {2010-01-01}, {2019-04-01}},',
r'        minor xtick={}, ',
f"        xticklabels={{`89, `00, `10, {ldate}}}, enlarge y limits={{lower, 0.3}}, ",
r'        enlarge x limits={0.05}]',
r'    \stdline{blue}{date}{value}{data/gdppc.csv}',
r'    \node[label={90:{\$'+ f'{gdppc_val:,}' + r'}}, circle, blue, fill, inner sep=1.5pt] at ',
f'        (axis cs:{date},{gdppc_val})' + r' {};',
r'    \end{axis}',
r'\end{tikzpicture}\\',
r'\footnotesize{Source: Bureau of Economic Analysis}']

f = text_dir.joinpath('gdp_chart.txt')
with f.open('w') as wf:
    for t in text:
        print(t)
        wf.write(t + '\n')

### Private fixed investment

In [ ]:
# Private fixed investment contribution to growth
s = ['A008RY', 'A011RY', 'A014RY']

df = nipa_df(retrieve_table('T10502')['Data'], s)

df.loc['1989':].to_csv(data_dir / 'inv.csv', index_label='date')

ldate = df.index[-1]
datetext = dtxt(ldate)["qtr1"]

tot_contr = cont_subt(df.loc[ldate, ['A008RY', 'A011RY']].sum())
bus_contr = cont_subt(df.loc[ldate, 'A008RY'], 'end')
res_contr = cont_subt(df.loc[ldate, 'A011RY'], 'end')
inv_contr = cont_subt(df.loc[ldate, 'A014RY'], 'end')

text = (f'During the quarter, private fixed investment {tot_contr} real GDP growth. '+
        f'Non-residential fixed investment {bus_contr}, while '+
        f'residential fixed investment {res_contr}. The change in private '+
        f'inventories {inv_contr}.')

write_txt(text_dir / 'inv_text2.txt', text)

In [ ]:
# In levels and share of GDP
s = ['A191RC', 'A007RC', 'A008RC', 'A011RC']

df = nipa_df(retrieve_table('T10105')['Data'], s)

ldate = df.index[-1]

inv_tot = df.loc[ldate, 'A007RC'] / 1_000_000
gdp_tot = df.loc[ldate, 'A191RC'] / 1_000_000
res_tot = df.loc[ldate, 'A011RC'] / 1_000
bus_tot = df.loc[ldate, 'A008RC'] / 1_000_000

inv_sh = (inv_tot / gdp_tot) * 100
res_sh = (res_tot / 1_000 / gdp_tot) * 100
bus_sh = (bus_tot / gdp_tot) * 100

text = (f'In {dtxt(ldate)["qtr2"]}, private fixed investment, '+
        f'which does not include inventory investment, totals \${inv_tot:.1f} trillion, '+
        f'equivalent to {inv_sh:.1f} percent of GDP. Non-residential (business) fixed '+
        f'investment totals \${bus_tot:.1f} trillion, or {bus_sh:.1f} percent of GDP, '+
        f'while residential fixed investment totals \${res_tot:.1f} billion ({res_sh:.1f} '+
        'percent of GDP).')

write_txt(text_dir / 'inv_text.txt', text)

### Imports and Exports

In [ ]:
# Import and Export share of GDP
s = ['B020RC', 'B021RC', 'B648RC', 'LA000006']
s2 = ['A191RC']
df = nipa_df(retrieve_table('T40205')['Data'], s)
df['A191RC'] = nipa_df(retrieve_table('T10105')['Data'], s2)
df['EX'] = df['B020RC'] - df['LA000006']
df['IM'] = df['B021RC'] - df['B648RC']
data = df.div(df['A191RC'], axis=0) * 100
data.loc['1989':].to_csv(data_dir / 'eximgdp.csv', index_label='date')

date = f'{qtrs[data.index[-1].quarter]} quarter of {data.index[-1].year}'
valex = data['EX'].iloc[-1]
valim = data['IM'].iloc[-1]

text = (f'Nonpetroleum goods and services imports (see {{\color{{green!60!teal!80!black}}'+
        f'\\textbf{{---}}}}) were equivalent to {valim:.1f} percent of GDP in the {date}, '+
        f'while exports of nonpetroleum goods and services (see {{\\color{{blue!90!cyan}}'+
        f'\\textbf{{---}}}}) were equivalent to {valex:.1f} percent of GDP.')

write_txt(text_dir / 'exim.txt', text)

text

### Goods Import Penetration

In [ ]:
s = ['A353RC']

G = nipa_df(retrieve_table('T10205')['Data'], s).sort_index()

s = ['A253RC', 'A255RC', 'B647RC', 'LA000004', 'A650RC', 'B651RC', 'A652RC', 'A653RC', 'B648RC']

MX = nipa_df(retrieve_table('T40205')['Data'], s).sort_index()

D = G['A353RC'] - MX['A253RC'] + MX['A255RC']
result = (MX['A255RC'] / D)

import_categories = ['B647RC', 'LA000004', 'A650RC', 'B651RC', 
                     'A652RC', 'A653RC', 'B648RC']
Msh = MX[import_categories].div(MX['A255RC'], axis=0)

Msh['Consumer'] = Msh['B647RC'] + Msh['A652RC'] + Msh['B651RC']
Msh['Capital'] = Msh['LA000004'] - Msh['B648RC'] + Msh['A650RC'] + Msh['A653RC']

final = Msh[['Consumer', 'Capital', 'B648RC']].multiply(result, axis=0) * 100

final.loc['1989':].to_csv(data_dir / 'goodsimpsh.csv', index_label='date')

ltdate = final.index[-1]
datetext = dtxt(ltdate)['qtr1']
cons = final['Consumer'].iloc[-1]
capi = final['Capital'].iloc[-1]
oil = final['B648RC'].iloc[-1]

text = (f'As of {datetext}, imports of consumer goods excluding petroleum '+
        f'and petroleum products are equivalent to {cons:.1f} percent of domestic '+
        'consumption of goods (see\cbox{cyan!40!white}). Petroleum-related imports '+
        f'claim {oil:.1f} percent (see\cbox{{purple}}) and imports of all other goods, '+
        f'primarily capital goods, industrial supplies, and materials are equivalent to '+
        f'{capi:.1f} percent (see\cbox{{blue!50!cyan}}).')

write_txt(text_dir / 'goodsimpsh1.txt', text)

ch11 = (final.loc['2011-01-01'] - final.iloc[0])

chlt = (final.iloc[-1] - final.loc['2011-01-01'])

chtxt = {}
for i, v in chlt.items():
    if v >= 0.1:
        chtxt[i] = f'increased by the equivalent of {v:.1f} percent'
    elif v <= -0.1:
        chtxt[i] = f'decreased by the equivalent of {abs(v):.1f} percent'
    else:
        chtxt[i] = 'was virtually unchanged'
        
text = ('From 1989 to 2011, imports of consumer goods excluding petroleum increased '+
        f'by the equivalent of {ch11["Consumer"]:.1f} percent of domestic '+
         'consumption of goods; petroleum and products imports '+
        f'increased by the equivalent of {ch11["B648RC"]:.1f} percent; '+
        'and all other goods increased by the equivalent '+
        f'of {ch11["Capital"]:.1f} percent. Since 2011, imports of '+
        f'consumer goods {chtxt["Consumer"]} of domestic goods demand; '+
        f'imports of petroleum and products {chtxt["B648RC"]}; and other '+
        f'imports {chtxt["Capital"]}.')

write_txt(text_dir / 'goodsimpsh.txt', text)

### ITA - Financial Account Balance

In [4]:
def bea_api_ita(ind_list, bea_key):
    ''' Return tables in table list for years in range'''
    import requests
    from datetime import datetime

    years = ','.join(map(str, range(1988, 2021)))

    api_results = []

    for ind in ind_list:
        url = f'https://www.bea.gov/api/data/?&UserID={bea_key}'\
              f'&method=GetData&datasetname=ITA&Indicator={ind}'\
              f'&Frequency=QSA&Year={years}&ResultFormat=json'

        r = requests.get(url)

        api_results.append((ind, r.text))

    return api_results

In [5]:
ind_list = ['FinAssetsExclFinDeriv', 'FinLiabsExclFinDeriv', 'FinDeriv', 'StatDisc']

api_results = bea_api_ita(ind_list, bea_key)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

results = pd.DataFrame({name: {i['TimePeriod']: i['DataValue'] 
                               for i in json.loads(series)['BEAAPI']['Results']['Data']} 
                        for name, series in api_results})
results.index = pd.to_datetime(results.index)
results = results.replace(r'^\s*$', np.nan, regex=True).astype('float').rolling(4).sum()
results['FAB'] = results['FinLiabsExclFinDeriv'] - results['FinAssetsExclFinDeriv']
results['TOT'] = results[['FAB', 'StatDisc', 'FinDeriv']].sum(axis=1)
final = (results.divide(gdp, axis=0).dropna(how='all') * 100).fillna(0)
final.loc['1989':, ['FAB', 'StatDisc', 'FinDeriv', 'TOT']].to_csv(data_dir / 'fab.csv', index_label='date')

s = final.iloc[-1]
liab = s.FinLiabsExclFinDeriv
assets = s.FinAssetsExclFinDeriv

ldate = dtxt(final.index[-1])['qtr1']

text = (f'Over the year ending {ldate}, net domestic acquisitions of foreign assets were '+
        f'equivalent to {assets:.1f} percent of GDP, while net domestic incurrence of foreign '+
        f'liabilities total {liab:.1f} percent of GDP. Domestic net borrowing totals '+
        f'{s.TOT:.1f} percent of GDP.')

write_txt(text_dir / 'fab.txt', text)

text

'Over the year ending 2020 Q1, net domestic acquisitions of foreign assets were equivalent to 5.0 percent of GDP, while net domestic incurrence of foreign liabilities total 7.2 percent of GDP. Domestic net borrowing totals 1.8 percent of GDP.'

### GDP Composition

In [ ]:
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY']

df = nipa_df(retrieve_table('T10502')['Data'], s).loc['1989':]
df.to_csv(data_dir / 'comp.csv', index_label='date')

ltdt = dtxt(df.index[-1])['qtr2']

sl = [('DPCERY', 'main'), ('A006RY', 'main'), 
      ('A822RY', 'end'), ('A019RY', 'end')]

d = {}

for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)
    
text = (f'In {ltdt}, consumer spending (see\cbox{{yellow!80!orange}}) '+
        f'{d["DPCERY"]} overall real GDP growth. Private domestic '+
        f'investment (see\cbox{{blue!70!black}}) {d["A006RY"]} real GDP '+
        'growth, government spending and investment (see\cbox{cyan!50!white}) '+
        f'{d["A822RY"]}, and net exports (see\cbox{{green!60!black}}) '+
        f'{d["A019RY"]}.')  

write_txt(text_dir / 'gdp_exp_basic.txt', text)
print(text)

### GDP per capita

In [ ]:
s = ['A939RC', 'A939RX']

df = nipa_df(retrieve_table('T70100')['Data'], s)
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1])  * df['A939RC'].iloc[-1]
df[['value']].loc['1989':].to_csv(data_dir / 'gdppc.csv', index_label='date')

### Domestic Investment

In [ ]:
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC']
df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']
df = df.drop(['A107RC', 'W056RC'], axis=1)

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
dft = df.diff()
dft = dft.div(dft['A261RC'], axis=0)
contr = dft.multiply((((df['A261RC'].pct_change() + 1) ** 4) - 1) * 100, axis=0)
contr.loc['1989':].to_csv(data_dir / 'gdi.csv', index_label='date')

### Consumer Spending and Residential Fixed Investment

In [ ]:
s = ['DNPIRY', 'DSERRY', 'DPCERY', 'DGDSRY', 'DMOTRY',
     'DFDHRY', 'DREQRY', 'DFXARY', 'DCLORY', 'DHLCRY',
     'DTRSRY', 'DRCARY', 'DFSARY', 'DIFSRY', 'DHUTRY',
     'A011RY']

n = {'TOTAL': '& Total',
     'DGDSRY': '\cbox{red} & Goods',
     'DMOTRY': '& \hspace{1mm} Motor Vehicles and Parts',
     'DFDHRY': '& \hspace{1mm} Furniture and HH Equipment',
     'DREQRY': '& \hspace{1mm} Recreational Durable Goods',
     'DFXARY': '& \hspace{1mm} Groceries',
     'DCLORY': '& \hspace{1mm} Clothes and Shoes',
     'OTHSERV': '\cbox{blue!75!white} & Services (ex. Shelter)',
     'DHLCRY': '& \hspace{1mm} Health Care Services',
     'DTRSRY': '& \hspace{1mm} Transportation',
     'DRCARY': '& \hspace{1mm} Recreational',
     'DFSARY': '& \hspace{1mm} Food and Accommodations',
     'DIFSRY': '& \hspace{1mm} Financial and Insurance',
     'SHELTER': '\cbox{green!85!blue} & Shelter ',
     'DHUTRY': '& \hspace{1mm} Housing Services and Utilities ',
     'A011RY': '& \hspace{1mm} Residential Fixed Investment'}


total = lambda x: x['DPCERY'] - x['DNPIRY']
othserv = lambda x: x['DSERRY'] - x['DNPIRY'] - x['DHUTRY']
shelter = lambda x: x['DHUTRY'] + x['A011RY']

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      .assign(TOTAL = total, OTHSERV = othserv, SHELTER = shelter)
      [list(n.keys())])

# Save timseries to csv
(df.loc['1989':, ['DGDSRY', 'OTHSERV', 'SHELTER']]
   .to_csv(data_dir / 'pce.csv', index_label='date', float_format='%g'))

# Save table
data = df.iloc[-5:].iloc[::-1].T

cols = [f'& {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = df.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = df.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = df.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
(data.round(2).applymap('{:,.2f}'.format)
 .to_csv(data_dir / 'pce.tex', sep='&', line_terminator='\\\ ', quotechar=' '))

In [ ]:
sl = [('TOTAL', 'main'), ('DGDSRY', 'main'), 
      ('OTHSERV', 'end'), ('SHELTER', 'end'), 
      ('DHLCRY', 'main')]

d = {}

for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)

t_3y = cont_subt(data.loc['& Total', '3-year'], 'of')
hc_3y = cont_subt(data.loc[n['DHLCRY'], '3-year'], 'end')
ld = dtxt(df.index[-1])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']
val3y = df.TOTAL.rolling(13).mean().iloc[-1].round(2)
vallt = df.TOTAL.iloc[-1]

compare = compare_text(vallt, val3y, [0.1, 0.5, 2.0])

txt1 = (f'These categories {d["TOTAL"]} GDP growth in {ld}, {compare} '+
        f'the average {t_3y} over the past three years. ')
write_txt(text_dir / 'pce1.txt', txt1)
print(txt1)

txt2 = (f'In the {ld2}, household spending on goods {d["DGDSRY"]} GDP growth, '+
         'household spending on services other than housing and utilities '+
        f'{d["OTHSERV"]}, and shelter spending and investment {d["SHELTER"]}. '+
        f'Spending on health care services {d["DHLCRY"]} GDP growth in {ld} and '+
        f'has {hc_3y}, on average, over the past three years.')
write_txt(text_dir / 'pce2.txt', txt2)
print(txt2)

### Trade Contribution to GDP

In [ ]:
s = ['A019RY', 'A253RY', 'A646RY', 'A255RY', 'A656RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'nx.csv', index_label='date')

sl = [('A253RY', 'main'), ('A646RY', 'end'), 
      ('A255RY', 'main'), ('A656RY', 'end')]

d = {}
for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)
    
ldate = dtxt(df.index[-1])['qtr2']

text = (f"Goods exports {d['A253RY']} GDP growth in {ldate} while "+
        f"services exports {d['A646RY']}. Good imports {d['A255RY']} "+
        f"GDP growth and services imports {d['A656RY']}.")

write_txt(text_dir / 'trade.txt', text)

### Business investment

In [ ]:
s = ['Y001RY', 'A009RY', 'Y033RY']

df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'businv.csv', index_label='date')

### Durable goods new orders

In [2]:
# New orders for capital goods excluding defense or aircraft
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/advm3'
param = 'cell_value,time_slot_id'
t = '&time=from+1992'
cat = '&category_code=NXA'
dtc = '&data_type_code=NO'
oth = '&for=us&seasonally_adj=yes'

url = f'{base}?get={param}&key={key}{cat}{dtc}{t}{oth}'

r = requests.get(url).json()

df = pd.DataFrame({'date': [pd.to_datetime(i[4]) for i in r[1:]], 
                   'value': [float(i[0]) for i in r[1:]]}).sort_values('date')

df = df.set_index('date')

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])

result = ((df['value'].resample('QS').sum() * 4  / gdp['A191RC']).dropna() * 100).iloc[1:]
(result.rename('value').loc['1989':].to_csv(data_dir / 'dgno.csv', index_label='date', header=True))

node = end_node(result, 'purple!50!violet')
write_txt(text_dir / 'dgno_node.txt', node)


ldate = dtxt(df.index[-1])['mon1']
date_latest = dtxt(df.index[-1])['datetime']
month_short = df.index[-1].strftime('%b')

In [3]:
one_yr_ch = (df.pct_change(12) * 100).dropna()['value'].iloc[-1]

if one_yr_ch >= 0.1:
    val = f'increased by {one_yr_ch:.1f} percent'
elif one_yr_ch <= -0.1:
    val = f'decreased by {abs(one_yr_ch):.1f} percent'
else:
    val = 'were virtually unchanged'

text = ('New orders for manufactured core capital goods excluding aircraft '+
        f'totalled \${df.iloc[-1][0] / 1000:,.0f} billion in {ldate}, '+
        f'equivalent to {result.iloc[-1]:.1f} percent of GDP '+
        '(see {\color{purple!50!violet}\\textbf{---}}). New orders '+
        f'{val} over the past year.')

write_txt(text_dir / 'dgno.txt', text)

text

'New orders for manufactured core capital goods excluding aircraft totalled \\$63 billion in May 2020, equivalent to 3.7 percent of GDP (see {\\color{purple!50!violet}\\textbf{---}}). New orders decreased by 3.6 percent over the past year.'

In [4]:
text = ('xtick={{1992-01-01}, {1995-01-01}, {2000-01-01}, {2005-01-01}, '+
f'{{2010-01-01}}, {{2015-01-01}}, {{{date_latest}}}}}, '+
f'xticklabels={{`92, `95, `00, `05, `10, `15, {month_short}}}, ')

text_full = ('\\begin{tikzpicture}'+
'\\begin{axis}[\\bbar{y}{0}, \dateaxisticks ytick={4, 6, 8}, width=6.5cm, height=5.4cm,'+
'ymin=2.9, clip=false,'+
text + 
'minor xtick={}]'+
'\\rebars'+
'\\thickline{purple!50!violet}{date}{value}{data/dgno.csv}'+
'\input{text/dgno_node.txt}'+
'\end{axis}'+
'\end{tikzpicture}')

write_txt(text_dir / 'dgno.tex', text_full)

### Government spending and investment

In [ ]:
n = {'A822RY': 'Total',
     'A823RY': '\hspace{1mm}Federal total',
     'A824RY': '\hspace{1mm}\cbox{blue!60!black}National defense',
     'A997RY': '\hspace{7mm}Consumption expenditures',
     'A788RY': '\hspace{7mm}Gross investment',
     'A825RY': '\hspace{1mm}\cbox{green!85!black}Nondefense',
     'A542RY': '\hspace{7mm}Consumption expenditures',
     'A798RY': '\hspace{7mm}Gross investment',
     'A829RY': '\hspace{-2mm}\cbox{purple!70!magenta}State \& local total',
     'A991RY': '\hspace{5mm}Consumption expenditures',
     'A799RY': '\hspace{5mm}Gross investment'}

s = n.keys()

df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'gov.csv', index_label='date')

d = series_info(df['A822RY'])    
d2 = {}
for i in ['A824RY', 'A825RY', 'A829RY']:
    d2[i] = cont_subt(df[i].iloc[-1], 'end')
    
ltmain = cont_subt(d['val_latest'], 'main')
compare = compare_text(d['val_latest'], d['one_year_mean'], [0.05, 0.25, 0.5])
compare2 = compare_text(d['val_latest'], d['mean'], [0.05, 0.25, 0.5])

ltoym = f'{d["one_year_mean"]:.2f} ' + ['percentage point' if abs(d["one_year_mean"]) < 1.05 else 'percentage points'][0]
ltmean = f'{d["mean"]:.2f} ' + ['percentage point' if abs(d["mean"]) < 1.05 else 'percentage points'][0]
    
txt = ('Government consumption expeditures and gross investment, which provide '+
       f'services and infrastructure, {ltmain} real GDP growth in {d["date_latest_ft"]}, '+
       f'{compare} the average contribution of {ltoym} '+
       f'over the past year, and {compare2} the average of {ltmean} since 1989. '+
       f'In {d["date_latest_ft"]}, federal defense spending and investment (see\cbox{{blue!60!black}}) '+
       f'{d2["A824RY"]}, nondefense federal government spending and investment '+
       f'(see\cbox{{green!85!black}}) {d2["A825RY"]}, and state and '+
       f'local government (see\cbox{{purple!70!magenta}}) {d2["A829RY"]}.')

write_txt(text_dir / 'gov.txt', txt)

txt

In [ ]:
result = df[n.keys()]

data = result.iloc[-5:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = result.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
data = data.applymap('{:.2f}'.format)
data.to_csv(data_dir / 'gov.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

### Government receipts and expenditures

In [ ]:
# State and local government
s = ['W024RC', 'W023RC']

df = nipa_df(retrieve_table('T30300')['Data'], s)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

df['GDP'] = gdp

(df.div(df['GDP'], axis=0) * 100).loc['1989':].to_csv(data_dir / 'slggdp.csv', index_label='date', float_format='%g')
df = df.dropna() / 1000000

In [ ]:
ldate = dtxt(df.index[-1])['qtr1']
gdp = df.GDP.iloc[-1]
gdp3y = df.GDP.iloc[-13]
gdpgr = ((gdp / gdp3y) - 1) * 100
exp = df.W024RC.iloc[-1]
exp3y = df.W024RC.iloc[-13]
expsh = exp / gdp * 100
exp3ysh = exp3y / gdp3y * 100
expgr = ((exp / exp3y) - 1) * 100
rec = df.W023RC.iloc[-1]
rec3y = df.W023RC.iloc[-13]
recsh = rec / gdp * 100
rec3ysh = rec3y / gdp3y * 100
recgr = ((rec / rec3y) - 1) * 100
diff = rec - exp
diffsh = diff / gdp * 100
diff3y = rec3y - exp3y
diffgr = ((diff / diff3y) - 1) * 100
if diff < 0:
    txt = 'deficit'
else:
    txt = 'surplus'

recshch = recsh - rec3ysh
pp = 'percentage points' if abs(recshch) > 1.05 else 'percentage point'
rectxt = f'{["increased" if recshch >= 0 else "decreased"][0]} by a total of {abs(recshch):.2f} {pp}'

expshch = expsh - exp3ysh
pp = 'percentage points' if abs(expshch) > 1.05 else 'percentage point'
exptxt = f'{["increased" if expshch >= 0 else "decreased"][0]} by a total of {abs(expshch):.2f} {pp}'

diffch = recshch - expshch

difftxt = [f'shrink by {diffch:.2f} percent of GDP' if diffch >= 0.1 
             else f'widen by {abs(diffch):.2f} percent of GDP' if diffch <= -0.1 
             else 'be unchanged'][0]


text = (f'Consolidated state and local government expenditures total \${exp:.1f} trillion, '+
        f'or {expsh:.1f} percent of GDP, in {ldate}, and receipts total '+
        f'\${rec:.1f} trillion, equivalent to {recsh:.1f} percent of GDP. The combined state'+
        f' and local government {txt} was '+
        f'\${abs(diff) * 1000:.0f} billion or {abs(diffsh):.2f} percent of GDP. '+
        f'Over the past three years, the expenditures to GDP ratio {exptxt} at the '+
        f'consolidated state and local level, and the ratio of '+
        f'receipts to GDP has {rectxt}, causing the {txt} to {difftxt}.')

write_txt(text_dir / 'govexprec1.txt', text)

text

In [ ]:
# Federal government
s = ['W005RC', 'W013RC']

df = nipa_df(retrieve_table('T30200')['Data'], s)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

df['GDP'] = gdp

(df.div(df['GDP'], axis=0) * 100).to_csv(data_dir / 'fedgdp.csv', index_label='date', float_format='%g')
df = (df / 1000000).dropna()

In [ ]:
ldate = dtxt(df.index[-1])['qtr1']
gdp = df.GDP.iloc[-1]
gdp3y = df.GDP.iloc[-13]
gdpgr = ((gdp / gdp3y) - 1) * 100
exp = df.W013RC.iloc[-1]
exp3y = df.W013RC.iloc[-13]
expsh = exp / gdp * 100
exp3ysh = exp3y / gdp3y * 100
expgr = ((exp / exp3y) - 1) * 100
rec = df.W005RC.iloc[-1]
rec3y = df.W005RC.iloc[-13]
recsh = rec / gdp * 100
rec3ysh = rec3y / gdp3y * 100
recgr = ((rec / rec3y) - 1) * 100
diff = rec - exp
diffsh = diff / gdp * 100
diff3y = rec3y - exp3y
diffgr = ((diff / diff3y) - 1) * 100
if diff < 0:
    txt = 'deficit'
else:
    txt = 'surplus'

recshch = recsh - rec3ysh
pp = 'percentage points' if abs(recshch) > 1.05 else 'percentage point'
rectxt = f'{["increased" if recshch >= 0 else "decreased"][0]} by a total of {abs(recshch):.1f} {pp}'

expshch = expsh - exp3ysh
pp = 'percentage points' if abs(expshch) > 1.05 else 'percentage point'
exptxt = f'{["increased" if expshch >= 0 else "decreased"][0]} by a total of {abs(expshch):.1f} {pp}'

diffch = recshch - expshch

difftxt = [f'shrink by {diffch:.1f} percent of GDP' if diffch >= 0.1 
             else f'widen by {abs(diffch):.1f} percent of GDP' if diffch <= -0.1 
             else 'be unchanged'][0]

text = (f'In {ldate}, federal government expenditures total \${exp:.1f} trillion, '+
        f'equivalent to {expsh:.1f} percent of GDP, and receipts total '+
        f'\${rec:.1f} trillion, or {recsh:.1f} percent of GDP. The federal {txt} was therefore '+
        f'\${abs(diff):.1f} trillion or {abs(diffsh):.1f} percent of GDP. '+
        f'Over the past three years, the ratio of expenditures to GDP {exptxt}, and the ratio of '+
        f'receipts to GDP has {rectxt}, causing the {txt} to {difftxt}.')

write_txt(text_dir / 'govexprec2.txt', text)

text

### Public Debt by Holder

In [ ]:
series = ['FDHBATN', 'GFDEBTN', 'FDHBFRBN', 'FDHBPIN', 'FDHBFIN']
start = '1988-01-01'
ftype = '&file_type=json'
base = 'https://api.stlouisfed.org/fred/series/observations?'

df = pd.DataFrame()

for srs in series:
    param = f'series_id={srs}&observation_start={start}&api_key={fred_key}'
   

    url = f'{base}{param}{ftype}'
    r = requests.get(url).json()['observations']
    data = pd.Series({i['date']: (float(i['value']) / 1000.0) 
                      if srs in series[:2] 
                      else float(i['value']) for i in r})
    
    df[srs] = data

In [ ]:
df = df.dropna()
df['PD'] = df['FDHBPIN'] - df['FDHBFIN']
df['IG'] = df['GFDEBTN'] - (df['FDHBFRBN'] + df['FDHBPIN'])

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

data = df.div(gdp / 1000.0, axis=0).dropna()

(data[['PD', 'FDHBFIN', 'FDHBFRBN', 'IG']] * 100).to_csv(data_dir / 'pubdebt.csv', index_label='date')

ld = pd.to_datetime(df.index[-1])
ldate = dtxt(ld)['qtr2']

sh = df.div(df['GFDEBTN'], axis=0).iloc[-1] * 100
lv = df.iloc[-1] / 1000
dl = data.iloc[-1] * 100

text = (f'In {ldate}, total public debt was \${lv.GFDEBTN:.1f} trllion, '+
        f'equivalent to {dl.GFDEBTN:.1f} percent of GDP. Of this, \${lv.PD:.1f} '+
        f'trillion, or {sh.PD:.1f} percent of the total, is held by '+
        'private domestic investors (see\cbox{green!60!black}). An additional '+
        f'\${lv.FDHBFIN:.1f} trillion, or {sh.FDHBFIN:.1f} percent '+
        'of the total, is held by foreign investors (see\cbox{orange!70!white}). '+
        'The remainder is held by the Federal Reserve (see\cbox{blue}) '+
        'and various government agencies and trusts (see\cbox{cyan!50!white}), '+
        'such as the Social Security Trust Fund. ')

write_txt(text_dir / 'pubdebt.txt', text)

text

### Personal Income

In [ ]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A065RC', 'A033RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC', 'A061RC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              TRANSFER = lambda x: x['A577RC'] - x['A061RC'])
      .drop(['A061RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
growth_contrib(df, 'A065RC').loc['1989':].to_csv(data_dir / 'pi.csv', index_label='date')


data = growth_contrib(df, 'A065RC').rename({'A065RC': 'TOTAL', 'A033RC': 'LABOR'}, axis=1)

val3y = data.rolling(12).mean().iloc[-1]

ltdate = dtxt(data.index[-1])['qtr1']


d = {}

for i in data.keys():
    val = data[i].iloc[-1]
    pp = 'percentage points' if abs(val) > 1.05 else 'percentage point'
    if val >= 0.1:
        d[i] = f'contributed {val:.2f} {pp} to'
    elif val <= 0.1:
        d[i] = f'subtracted {abs(val):.2f} {pp} from'
    else:
        d[i] = 'did not contribute significantly to'
        
d['TOTAL'] = (d['TOTAL']
              .replace('contributed', 'increased at an annualized rate of')
              .replace('subtracted', 'decreased at an annualized rate of')
              .replace('percentage points', 'percent')
              .replace(' to', '').replace(' from', ''))

for i in ['TRANSFER', 'CAPITAL']:
    d[i] = d[i].replace(' to', '').replace(' from', '')
    
    
d2 = {}

for i in data.keys():
    val = val3y[i]
    pp = 'percentage points' if abs(val) > 1.05 else 'percentage point'
    if val >= 0.1:
        d2[i] = f'contributed an average of {val:.2f} {pp}'
    elif val <= 0.1:
        d2[i] = f'subtracted an average of {abs(val):.2f} {pp}'
    else:
        d2[i] = 'did not contribute significantly, on average'
        
text = (f'Aggregate real personal income {d["TOTAL"]} in {ltdate}. '+
        f'Labor income {d["LABOR"]} overall growth, '+
        f'capital income {d["CAPITAL"]}, and welfare income {d["TRANSFER"]}. ')

write_txt(text_dir / 'pi.txt', text)

text

### Consumer Spending Growth Contributions

In [ ]:
s = ['B230RC']

population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']

s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A067RC', 'A068RC', 'A071RC', 'DPCERC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(OTHER = lambda x: -(x['A068RC'] - x['DPCERC']),
              SAVING = lambda x: -x['A071RC'])
      .drop(['A068RC'], axis=1)
      .divide(population, axis=0)
      .multiply(deflator, axis=0))

data = growth_contrib(df, 'DPCERC').rolling(4).mean()
data3y = growth_contrib(df, 'DPCERC').rolling(12).mean()
data.loc['1989':].to_csv(data_dir / 'pcedecomp.csv', index_label='date')

In [ ]:
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'

pce = round(data['DPCERC'].iloc[-1], 1)
if pce > 0:
    pcedir = 'increased'
elif pce < 0:
    pcedir = 'decreased'
else: 
    pcedir = ''
pcetext = f'{pcedir} at an average rate of {abs(pce):.1f} percent' 
if pce == 0:
    pcetext = 'was unchanged'
    
slist = ['A067RC', 'SAVING', 'OTHER']
d = {}
for i in slist:
    d[i] = round(data[i].iloc[-1], 1)
    tname = f'{i}txt'
    if d[i] > 0:
        tmpdir = 'added'
    elif d[i] < 0:
        tmpdir = 'subtracted'
    else:
        tmpdir = ''
    tmptxt = f'{tmpdir} {abs(d[i]):.1f} percentage points'
    if d[i] == 0:
        tmptxt = "didn't affect the total"
    d[tname] = tmptxt

In [ ]:
pcetxt1 = (f'Real per capita consumer spending {pcetext} over the '+
           f'four quarters ending {date}. Changes to disposable income '+
           f'{d["A067RCtxt"]}, changes to saving {d["SAVINGtxt"]}, and '+
           f'changes to other outlays {d["OTHERtxt"]}.')

pcetxt2 = ('Over the past three years, real per capita consumer spending '+
           f'growth has averaged {data3y["DPCERC"].iloc[-1]:.1f} percent, '+
           f'with income growth contribuing an average of {data3y["A067RC"].iloc[-1]:.1f} '+
           'percentage points and saving subtracting an average of '+
           f'{abs(data3y["SAVING"].iloc[-1]):.1f} percentage points.')

pcetxt = f'{pcetxt1} {pcetxt2}'

write_txt(text_dir / 'pcedecomp.txt', pcetxt)

print(pcetxt1)

print(pcetxt2)

### Sectoral Accounts

**NOTE:** Need to convert "deficit", "borrower" etc to parameters!

In [ ]:
s = ['W162RC', 'W994RC', 'AD01RC', 'W995RC', 'W996RC', 'AD03RC']
df = (nipa_df(retrieve_table('T50100')['Data'], s).div(
      nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'], axis=0)) * 100

df[['W995RC', 'W996RC', 'AD03RC']].dropna().to_csv(data_dir / 'sectbal2.csv', index_label='date')

data = pd.DataFrame()
data['PRIV'] = df['W994RC']
data['GOV'] = df['AD01RC']
data['ROW'] = -df['W162RC']
data = data.dropna()
date = dtxt(data.index[-1])['qtr1']

data.dropna().to_csv(data_dir / 'sectbal.csv', index_label='date')

priv_curr = abs(data['PRIV'].iloc[-1])
priv_prev = abs(data.loc['2015-01-01', 'PRIV'])
gov_curr = abs(data['GOV'].iloc[-1])
gov_prev = abs(data.loc['2015-01-01', 'GOV'])
row_curr = abs(data['ROW'].iloc[-1])
row_prev = abs(data.loc['2015-01-01', 'ROW'])

In [ ]:
compare = compare_text(priv_curr, priv_prev, [0.4, 1.0, 3.5])

sectbaltxt = (f"In {date}, the US private sector was a net lender (running a surplus) of "+
              f"the equivalent of {priv_curr:.1f} percent of GDP, {compare} the {priv_prev:.1f} "+
              "percent surplus in 2015 Q1. The rest of the world was a net lender to the US, to the "+
              f"equivalent of {row_curr:.1f} percent of GDP in {date} compared to {row_prev:.1f} "+
              f"percent in 2015 Q1. Balancing these transactions, the government (federal, state, "+
              f"and local combined) was a net borrower (running a deficit) of the equivalent "+
              f"of {gov_curr:.1f} percent of GDP, compared to {gov_prev:.1f} percent in 2015. ")
print(sectbaltxt)

write_txt(text_dir / 'sectbal.txt', sectbaltxt)

In [ ]:
dt = df.dropna().index[-1]
ltdate = dtxt(dt)['qtr1']
hhsh = df.loc[dt, 'W996RC']
pbussh = f"{df.loc[dt, 'W995RC']:.1f} percent of GDP"

if (df.loc[dt, 'W995RC'] > -0.1) & (df.loc[dt, 'W995RC'] < 0.1):
    pbussh = 'unchanged'
    
text = (f'Within the private sector, households were net lenders of the equivalent of {hhsh:.1f} '+
        f'percent of GDP in {ltdate} (see\cbox{{orange!90!yellow}}), while the '+
         'net financial balance of private businesses--corporate and noncorporate--was '+
        f'equivalent to {pbussh} (see\cbox{{purple!50!red}}).')  

write_txt(text_dir / 'sectbal2.txt', text)

text

### Current Account Balance

In [37]:
df

,A1073C,B020RC,A253RC,A332RC,A339RC,A646RC,B645RC,LA000035,W163RC,B021RC,...,A333RC,A340RC,B656RC,A655RC,A123RC,A124RC,GOODS,SERVICES,INCOME,TRANSFERS
1988-01-01,564203.0,418727.0,311241.0,198655.0,112586.0,107486.0,145471.0,NaN,691070.0,542714.0,...,284283.0,157204.0,101227.0,120482.0,27876.0,-126867.0,-130246.0,6259.0,24989.0,-27876.0
1988-04-01,585946.0,439471.0,328498.0,210291.0,118207.0,110973.0,146475.0,NaN,695902.0,546098.0,...,289534.0,156145.0,100419.0,123659.0,26144.0,-109956.0,-117181.0,10554.0,22816.0,-26144.0
1988-07-01,607036.0,453586.0,338686.0,213165.0,125521.0,114900.0,153453.0,NaN,713492.0,552849.0,...,293799.0,157322.0,101728.0,133291.0,27353.0,-106456.0,-112435.0,13172.0,20162.0,-27353.0
1988-10-01,629683.0,466619.0,349792.0,225546.0,124246.0,116828.0,163066.0,NaN,749299.0,574314.0,...,311495.0,158718.0,104100.0,140403.0,34583.0,-119615.0,-120422.0,12728.0,22663.0,-34583.0
1989-01-01,656627.0,485189.0,360484.0,232456.0,128028.0,124705.0,171439.0,NaN,763318.0,586154.0,...,312446.0,169540.0,104168.0,149043.0,28131.0,-106691.0,-121502.0,20537.0,22396.0,-28131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01,3817193.0,2520278.0,1661761.0,1010132.0,651630.0,858517.0,1149031.0,147884.0,4363775.0,3154126.0,...,1668906.0,885432.0,599788.0,891151.0,318499.0,-546583.0,-892576.0,258729.0,257880.0,-318499.0
2019-04-01,3827770.0,2504031.0,1646077.0,976197.0,669880.0,857954.0,1177190.0,146548.0,4341877.0,3166691.0,...,1644912.0,921180.0,600600.0,876194.0,298991.0,-514107.0,-920014.0,257354.0,300996.0,-298991.0
2019-07-01,3806069.0,2495136.0,1638000.0,973496.0,664504.0,857136.0,1160355.0,150578.0,4313000.0,3148168.0,...,1648601.0,892321.0,607245.0,851368.0,313464.0,-506930.0,-902922.0,249891.0,308987.0,-313464.0
2019-10-01,3786950.0,2497726.0,1629201.0,966542.0,662659.0,868525.0,1148746.0,140478.0,4230836.0,3075592.0,...,1592600.0,866384.0,616608.0,834512.0,320731.0,-443886.0,-829783.0,251917.0,314234.0,-320731.0


In [38]:
s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

n = {'A1073C': 'Current receipts',
     'B020RC': '\hspace{1mm}Exports',
     'A253RC': '\hspace{3mm}Goods',
     'A332RC': '\hspace{5mm}Durable',
     'A339RC': '\hspace{5mm}Nondurable',
     'A646RC': '\hspace{3mm}Services',
     'B645RC': '\hspace{1mm}Income receipts',
     'LA000035': '\hspace{1mm}Transfer receipts',
     'W163RC': 'Current payments',
     'B021RC': '\hspace{1mm}Imports',
     'A255RC': '\hspace{3mm}Goods',
     'A333RC': '\hspace{5mm}Durable',
     'A340RC': '\hspace{5mm}Nondurable',
     'B656RC': '\hspace{3mm}Services',
     'A655RC': '\hspace{1mm}Income payments',
     'A123RC': '\hspace{1mm}Transfer payments',
     'A124RC': 'Current account balance'}


#s = ['A124RC', 'A253RC', 'A255RC', 'A646RC', 'B656RC', 'B645RC',
#     'A655RC', 'A123RC']

s = ['A124RC', 'GOODS', 'SERVICES', 'INCOME', 'TRANSFERS']

df = (nipa_df(retrieve_table('T40100')['Data'], n.keys())
      .assign(GOODS = lambda x: x['A253RC'] - x['A255RC'],
              SERVICES = lambda x: x['A646RC'] - x['B656RC'],
              INCOME = lambda x: x['B645RC'] - x['A655RC'],
              TRANSFERS = lambda x: - x['A123RC']))

data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0).loc['1989':].multiply(100).round(2))

data[s].dropna().to_csv(data_dir / 'cab.csv', index_label='date')

cab = abs(data['A124RC'].iloc[-1])
tb = abs(data['GOODS'].iloc[-1])
ld = dtxt(data.index[-1])['qtr1']

text = (f'As of {ld}, the US runs a current account deficit of {cab:.1f} '+
        'percent of GDP, primarily as the result of a trade deficit on '+
        f'goods of {tb:.1f} percent of GDP.')

write_txt(text_dir / 'cab.txt', text)

text

'As of 2020 Q1, the US runs a current account deficit of 2.0 percent of GDP, primarily as the result of a trade deficit on goods of 3.6 percent of GDP.'

In [39]:
data

,A1073C,B020RC,A253RC,A332RC,A339RC,A646RC,B645RC,LA000035,W163RC,B021RC,...,A333RC,A340RC,B656RC,A655RC,A123RC,A124RC,GOODS,SERVICES,INCOME,TRANSFERS
1989-01-01,11.91,8.80,6.54,4.22,2.32,2.26,3.11,NaN,13.85,10.64,...,5.67,3.08,1.89,2.70,0.51,-1.94,-2.20,0.37,0.41,-0.51
1989-04-01,12.24,9.04,6.76,4.32,2.44,2.27,3.20,NaN,13.91,10.61,...,5.61,3.13,1.87,2.82,0.48,-1.67,-1.98,0.41,0.39,-0.48
1989-07-01,12.08,8.94,6.66,4.34,2.32,2.29,3.14,NaN,13.51,10.26,...,5.35,3.03,1.88,2.71,0.54,-1.43,-1.73,0.41,0.43,-0.54
1989-10-01,12.12,8.97,6.61,4.32,2.29,2.36,3.15,NaN,13.64,10.41,...,5.38,3.13,1.90,2.61,0.62,-1.52,-1.90,0.46,0.54,-0.62
1990-01-01,12.29,9.16,6.75,4.46,2.29,2.42,3.13,NaN,13.83,10.67,...,5.26,3.43,1.98,2.63,0.53,-1.53,-1.94,0.44,0.50,-0.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01,18.09,11.95,7.88,4.79,3.09,4.07,5.45,0.70,20.68,14.95,...,7.91,4.20,2.84,4.22,1.51,-2.59,-4.23,1.23,1.22,-1.51
2019-04-01,17.94,11.73,7.71,4.57,3.14,4.02,5.52,0.69,20.35,14.84,...,7.71,4.32,2.81,4.11,1.40,-2.41,-4.31,1.21,1.41,-1.40
2019-07-01,17.67,11.58,7.60,4.52,3.08,3.98,5.39,0.70,20.02,14.61,...,7.65,4.14,2.82,3.95,1.46,-2.35,-4.19,1.16,1.43,-1.46
2019-10-01,17.43,11.49,7.50,4.45,3.05,4.00,5.29,0.65,19.47,14.15,...,7.33,3.99,2.84,3.84,1.48,-2.04,-3.82,1.16,1.45,-1.48


In [30]:
result = data[n.keys()]


In [31]:
result.iloc[-5:].iloc[::-1].T

,2020-01-01,2019-10-01,2019-07-01,2019-04-01,2019-01-01
A1073C,16.72,17.43,17.67,17.94,18.09
B020RC,11.24,11.49,11.58,11.73,11.95
A253RC,7.42,7.50,7.60,7.71,7.88
A332RC,4.35,4.45,4.52,4.57,4.79
A339RC,3.08,3.05,3.08,3.14,3.09
A646RC,3.82,4.00,3.98,4.02,4.07
B645RC,4.82,5.29,5.39,5.52,5.45
LA000035,0.66,0.65,0.70,0.69,0.70
W163RC,18.75,19.47,20.02,20.35,20.68
B021RC,13.68,14.15,14.61,14.84,14.95


In [40]:
result = data[n.keys()]
data2 = result.iloc[-6:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data2.columns)]

data2.columns = cols
data2['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data2['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data2.index = data2.index.map(n)
data2 = data2.applymap('{:.2f}'.format)
data2.to_csv(data_dir / 'cab.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

In [41]:
data2

,2020 Q1,`19 Q4,`19 Q3,`19 Q2,`19 Q1,`18 Q4,3-year,10-year
Current receipts,16.72,17.43,17.67,17.94,18.09,18.17,17.88,18.32
\hspace{1mm}Exports,11.24,11.49,11.58,11.73,11.95,12.01,11.93,12.64
\hspace{3mm}Goods,7.42,7.50,7.60,7.71,7.88,7.94,7.84,8.51
\hspace{5mm}Durable,4.35,4.45,4.52,4.57,4.79,4.79,4.76,5.26
\hspace{5mm}Nondurable,3.08,3.05,3.08,3.14,3.09,3.15,3.08,3.25
\hspace{3mm}Services,3.82,4.00,3.98,4.02,4.07,4.07,4.09,4.13
\hspace{1mm}Income receipts,4.82,5.29,5.39,5.52,5.45,5.39,5.20,4.94
\hspace{1mm}Transfer receipts,0.66,0.65,0.70,0.69,0.70,0.76,0.75,0.74
Current payments,18.75,19.47,20.02,20.35,20.68,20.97,20.25,20.80
\hspace{1mm}Imports,13.68,14.15,14.61,14.84,14.95,15.29,14.88,15.73


In [19]:
result.iloc[-5:].iloc[::-1].T

,2020-01-01,2019-10-01,2019-07-01,2019-04-01,2019-01-01
1988-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1988-04-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1988-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1988-10-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1989-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
B021RC,NaN,NaN,NaN,NaN,NaN
B645RC,NaN,NaN,NaN,NaN,NaN
B656RC,NaN,NaN,NaN,NaN,NaN
LA000035,NaN,NaN,NaN,NaN,NaN


In [2]:
nipa_series_codes(retrieve_table('T40100'))

{'A1073C': 'Current receipts from the rest of the world',
 'B020RC': 'Exports of goods and services',
 'A253RC': 'Goods',
 'A332RC': 'Durable',
 'A339RC': 'Nondurable',
 'A646RC': 'Services',
 'B645RC': 'Income receipts',
 'B4188C': 'Wage and salary receipts',
 'W160RC': 'Income receipts on assets',
 'A2067C': 'Interest',
 'B3375C': 'Dividends',
 'B3475C': 'Reinvested earnings on U.S. direct investment abroad',
 'LA000035': 'Current taxes, contributions for government social insurance, and transfer receipts from the rest of the world',
 'LA000036': 'To persons',
 'LA000029': 'To business',
 'LA000037': 'To government',
 'W163RC': 'Current payments to the rest of the world',
 'B021RC': 'Imports of goods and services',
 'A255RC': 'Goods',
 'A333RC': 'Durable',
 'A340RC': 'Nondurable',
 'B656RC': 'Services',
 'A655RC': 'Income payments',
 'B4189C': 'Wage and salary payments',
 'W161RC': 'Income payments on assets',
 'B1869C': 'Interest',
 'B3376C': 'Dividends',
 'B3476C': 'Reinvested earn

### Federal Interest Outlays share of GDP

In [ ]:
tot = fred_df('FYOINT')['VALUE']
fy = tot.index[-1].year
pfy = tot.index[-2].year
lttot = tot.iloc[-1] / 1000
prtot = tot.iloc[-2] / 1000

data = fred_df('FYOIGDA188S')['VALUE']

data.to_csv(data_dir / 'fedintexp.csv', index_label='date', header=True)

latest = data.iloc[-1]
pval = data.iloc[-2]
val90s = data.loc['1990':'1999'].mean()


text = (f'The Office of Management and Budget '+
        '\href{https://www.whitehouse.gov/omb/historical-tables/}{reports} '+
        f'federal interest outlays of \${lttot:.1f} billion in fiscal year {fy}, '+
        f'compared to \${prtot:.1f} billion in fiscal year {pfy}. '+
        f'Put into the context of the size of the economy, federal interest '+
        f'outlays in {fy} were equivalent to {latest:.1f} percent of '+
        f'GDP, {pval:.1f} percent of GDP in {pfy}, and an average of {val90s:.1f} '+
        'percent in the 1990s, when interest rates were substantially higher. ')

write_txt(text_dir / 'fedintexp.txt', text)

print(text)

node = end_node(data, 'magenta')
write_txt(text_dir / 'fedintexp_node.txt', node)

In [ ]:
#nipa_series_codes(retrieve_table('T20100'))

### Corporate Profits Destination

In [ ]:
s = ['A032RC', 'A438RC', 'A054RC', 'B056RC', 'A127RC']
cprof = (nipa_df(retrieve_table('T11200')['Data'], s) / 1000000).dropna()

cprof['NNI'] = cprof['A032RC'] - cprof['A438RC']
cprof['TAX'] = cprof['A054RC'] / cprof['NNI'] * 100 
cprof['DIV'] = cprof['B056RC'] / cprof['NNI'] * 100 
cprof['RE'] = cprof['A127RC'] / cprof['NNI'] * 100 

cprof[['TAX', 'DIV', 'RE']].loc['1989':].to_csv(data_dir / 'cprof.csv', index_label='date')

dt = cprof.index[-1]
ltdate = dtxt(cprof.index[-1])['qtr2']
tot = cprof.loc[dt, ['A054RC', 'B056RC', 'A127RC']].sum()
totsh = tot / cprof.loc[dt, 'NNI'] * 100
div = cprof.loc[dt, 'B056RC']
divsh = div / cprof.loc[dt, 'NNI'] * 100
ret = cprof.loc[dt, 'A127RC']
tax = cprof.loc[dt, 'A054RC']

text = (f'In {ltdate}, aggregate corporate profits were \${tot:.2f} trillion, or {totsh:.1f} '+
        f'percent of net national income. Of this, \${div:.2f} trillion, equivalent to {divsh:.1f} '+
        'percent of net national product, were paid out as dividends (see\cbox{blue!70!purple}), '+
        f'\${ret*1000:.0f} billion were retained (see\cbox{{cyan!50!white}}), and \${tax*1000:.0f} billion '+
        'went to corporate income tax (see\cbox{red!80!orange}). ')

write_txt(text_dir / 'cprof.txt', text)

text

### Corporate profits source

In [ ]:
s = ['W170RC', 'A262RC', 'W986RC', 'A922RC']
df1 = nipa_df(retrieve_table('T50100')['Data'], s)

s = ['A123RC']
df2 = nipa_df(retrieve_table('T40100')['Data'], s)

s = ['A001RC']
df3 = nipa_df(retrieve_table('T10705')['Data'], s)

cprof = pd.DataFrame()
cprof['ROW Saving'] = (df2['A123RC'] / df3['A001RC']) * 100
cprof['HH Saving'] = (- df1['W986RC'] / df3['A001RC']) * 100
cprof['Gov Saving'] = (- df1['A922RC'] / df3['A001RC']) * 100
cprof['Investment'] = ((df1['W170RC'] - df1['A262RC']) / df3['A001RC']) * 100

cprof.loc['1989':].to_csv(data_dir / 'cprof2.csv', index_label='date')

In [ ]:
#nipa_series_codes(retrieve_table('T50100'))

### Labor Productivity

In [10]:
# Series stored as a dictionary
series = {'PRS85006092': 'value',
          'PRS85006032': 'hours',
          'PRS85006042': 'output',
          'PRS84006033': 'hours_index'}

In [11]:
# Start year and end year
dates = (1989, 2020)
df = bls_api(series, dates, bls_key)

df.to_csv(data_dir / 'lprod.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [12]:
df = pd.read_csv(data_dir / 'lprod.csv')
df = df.set_index(pd.to_datetime(df['date']))
df = df.drop('date',axis=1)

d = series_info(df['value'])

s = {}

for srs in list(series.values()):
    s[srs] = {}
    tmp = series_info(df[srs])
    for i in ['val_latest', 'val_prev']:
        if tmp[i] > 0:
            s[srs][i] = f'increased at an annual rate of {tmp[i]:.1f} percent'
            s[srs][i+'2'] = f'an increase of {tmp[i]:.1f} percent'
        elif tmp[i] < 0:
            s[srs][i] = f'decreased at an annual rate of {abs(tmp[i]):.1f} percent'
            s[srs][i+'2'] = f'a decrease of {abs(tmp[i]):.1f} percent'
        else:
            s[srs][i] = 'was unchanged'
            s[srs][i+'2'] = 'no change'   
            
compare = compare_text(d["five_year_mean"], d["mean"], [0.1, 0.5, 2.0])
    

text = (f'In {d["date_latest_ft"]}, labor productivity {s["value"]["val_latest"]} '+
        f'(see\cbox{{teal}}), as the result of {s["output"]["val_latest2"]} in real ouput and '+
        f'{s["hours"]["val_latest2"]} in hours worked. In the prior quarter, '+
        f'{d["date_prev_ft"]}, labor productivity {s["value"]["val_prev"]}, as '+
        f'real output {s["output"]["val_prev"]} and hours of work {s["hours"]["val_prev"]}. '+
        f'Over the past five years, '+
        f'labor productivity growth has averaged {d["five_year_mean"]:.1f} percent, '+
        f'{compare} the 1989-onward average of {d["mean"]:.1f} percent.')

write_txt(text_dir / 'lprod.txt', text)

text

'In 2020 Q1, labor productivity decreased at an annual rate of 0.9 percent (see\\cbox{teal}), as the result of a decrease of 6.5 percent in real ouput and a decrease of 5.6 percent in hours worked. In the prior quarter, 2019 Q4, labor productivity increased at an annual rate of 1.2 percent, as real output increased at an annual rate of 2.4 percent and hours of work increased at an annual rate of 1.2 percent. Over the past five years, labor productivity growth has averaged 1.1 percent, substantially below the 1989-onward average of 1.9 percent.'

In [13]:
text = (f'Total hours worked in nonfarm businesses {s["hours"]["val_latest"]} '+
        f'in {d["date_latest_ft"]}, following {s["hours"]["val_prev2"]} in '+
        f'{d["date_prev_ft"]}.')

write_txt(text_dir / 'tot_hours.txt', text)

text

'Total hours worked in nonfarm businesses decreased at an annual rate of 5.6 percent in 2020 Q1, following an increase of 1.2 percent in 2019 Q4.'

### Gross Labor Income

In [8]:
# Series stored as a dictionary
series = {'LNS12005054': 'Hours', 'LNS12000000': 'Employment'}

# Start year and end year
dates = (1989, 2020)
emp_hrs = bls_api(series, dates, bls_key)

emp = pd.read_csv(data_dir / 'lprod.csv').set_index('date')['hours_index']

Post Request Status: REQUEST_SUCCEEDED


In [9]:
emp_hrs['Total'] = emp_hrs['Hours'] * emp_hrs['Employment']
emp = emp_hrs['Total'].resample('QS').mean()

s = ['A033RC']
coe = nipa_df(retrieve_table('T20100')['Data'], s)

s = ['DPCERG']
pce = nipa_df(retrieve_table('T20304')['Data'], s)

data = coe.join(pce).join(emp).dropna()
data['real_coe'] = data['A033RC'] / (data['DPCERG'] / data['DPCERG'].iloc[-1])
data['coe_inp'] = data['real_coe'] / data['Total']
data['wage'] = data['coe_inp'] * data['Total'].iloc[0]
data['work'] = data['real_coe'] - data['wage']
# Calculate contributions to growth
result = growth_contrib(data, 'real_coe')[['work', 'wage']]
result.dropna().loc['1989':].to_csv(data_dir / 'gli.csv', index_label='date')

date = f'{result.index[-1].year} Q{result.index[-1].quarter}'
totval = result.iloc[-1].sum()
wage = result['wage'].iloc[-1]
work = result['work'].iloc[-1]

if totval > 0:
    txt1 = f'increased at an annualized and inflation-adjusted rate of {totval:.2f} percent in {date}.'
elif totval < 0:
    txt1 = f'decreased at an annualized and inflation-adjusted rate of {abs(totval):.2f} percent in {date}.'
else:
    txt1 = f'was unchanged, after adjusting for inflation, in {date}.'
    
if wage > 0:
    txt2 = f'Changes in wages contributed {wage:.2f} percentage points'
elif wage < 0:
    txt2 = f'Changes in wages subtracted {abs(wage):.2f} percentage points'
else: 
    txt2 = 'Changes in wages did not contribute'
    
if work > 0:
    txt3 = f', and changes in total hours worked contributed {work:.2f} percentage points.'
elif work < 0:
    txt3 = f', amd changes in total hours worked subtracted {abs(work):.2f} percentage points.'
else: 
    txt3 = ', and changes in total hours worked did not contribute.'    
    
text = f'{txt1} {txt2}{txt3}'
write_txt(text_dir / 'gli.txt', text)

text

'decreased at an annualized and inflation-adjusted rate of 1.13 percent in 2020 Q1. Changes in wages contributed 3.34 percentage points, amd changes in total hours worked subtracted 4.47 percentage points.'

In [ ]:
#data

### Financial Obligations Ratio

In [7]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=FOR&series=6d253f59629f788ab250a7a30662e79e&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d = {k: v.replace(', seasonally adjusted', '') for k, v in d.items()}

data = clean_data.rename(d, axis=1)['Financial obligations ratio']
data.to_csv(data_dir / 'for.csv', index_label='date')

node = end_node(data, 'blue!80!black')
write_txt(text_dir / 'for_node.txt', node)

ltdate = dtxt(data.index[-1])['qtr1']
ltval = data.iloc[-1]

text = (f'As of {ltdate} the financial obligations ratio is {ltval:.1f} percent')

write_txt(text_dir / 'for.txt', text)

text

'As of 2020 Q1 the financial obligations ratio is 15.0 percent'

### Shiller real return trailing 20-year average

In [5]:
data = pd.read_excel('http://www.econ.yale.edu/~shiller/data/ie_data.xls', sheet_name='Data')
df1 = data.iloc[679:1801, np.r_[0, 14]]
df1.columns = ['Date', 'CATRPE']
df1.index = pd.to_datetime([f'{val:.2f}' for val in df1['Date'].values])
df1 = df1.loc[~df1.index.duplicated(keep='first')]
df1.loc['1989':, 'CATRPE'].to_csv(data_dir / 'catrpe.csv', index_label='date')

node = end_node(df1.CATRPE, 'blue!80!black', percent=False, date=True)
write_txt(text_dir / 'cape_node.txt', node)

df = data.iloc[679:1801, np.r_[0, 7, 8]]
df.columns = ['Date', 'Price', 'Dividend']
df.index = pd.to_datetime([f'{val:.2f}' for val in df['Date'].values])
df = df.loc[~df.index.duplicated(keep='first')]

In [6]:
for yrs in [10, 15, 20]:
    s = df.copy()
    mos = yrs * 12
    s['DY'] = (s['Dividend'] / s['Price']).rolling(mos).mean()
    s['Pch'] = s['Price'].pct_change(mos)
    s['Pch'] = (s['Pch']+1)**(1/yrs) - 1
    s = s.dropna()
    s['Return'] = (s['DY'] + s['Pch']) * 100
    s.loc['1989':].dropna().to_csv(data_dir / f'sp500rr{yrs}.csv', 
                                   index_label='date', float_format='%g')
    
text = ('According to historical stock market '+
        '\href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, '+
        'the inflation-adjusted trailing twenty year annual rate of return '+
        f'of the S\&P 500 was {s["Return"].iloc[-1]:.1f} percent as '+
        f'of {s.index[-1].strftime("%B %Y")}. Real returns are currently low relative '+
        'to the average trailing twenty year real annual return of '+
        f'{s["Return"].loc["1995":"2005"].mean():.1f} percent during 1995--2005.')

write_txt(text_dir / 'sp500rr3.txt', text)

text

'According to historical stock market \\href{www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, the inflation-adjusted trailing twenty year annual rate of return of the S\\&P 500 was 2.9 percent as of March 2020. Real returns are currently low relative to the average trailing twenty year real annual return of 10.1 percent during 1995--2005.'

### High Quality Corporate Bond Yield

In [6]:
df = fred_df('HQMCB10YR')
df.to_csv(data_dir / 'hqcb.csv', index_label='date')
node = end_node(df['VALUE'], 'blue!60!violet', date=True)
write_txt(text_dir / 'hqcb_node.txt', node)

In [7]:
ltval = df.VALUE.iloc[-1]
prval = df.VALUE.iloc[-2]
prval2 = df.VALUE.iloc[-13]
ltdate = dtxt(df.index[-1])['mon1']
prdate = dtxt(df.index[-2])['mon1']
prdate2 = dtxt(df.index[-13])['mon1']

text = (f'This monthly measure shows a spot rate of {ltval:.1f} '+
        f'percent in {ltdate} for high-quality corporate bonds with '+
        f'a maturity of 10 years, following a rate of {prval:.1f} percent '+
        f'in {prdate}. In {prdate2}, the spot rate was {prval2:.1f} percent.')

write_txt(text_dir / 'hqcb.txt', text)

text

'This monthly measure shows a spot rate of 2.6 percent in May 2020 for high-quality corporate bonds with a maturity of 10 years, following a rate of 2.8 percent in April 2020. In May 2019, the spot rate was 3.6 percent.'

### International Investment Position (IIP)

In [2]:
series = ['IIPUSLIAA', 'IIPUSASSA','GDPA']
df = pd.DataFrame()
for s in series:
    df[s] = fred_df(s)['VALUE']
    
series2 = ['IIPUSLIAQ', 'IIPUSASSQ', 'GDP']
df2 = pd.DataFrame()
for s in series2:
    df2[s] = fred_df(s)['VALUE']
    
data = pd.DataFrame()
data['Liab'] = (df.loc[:'2005', 'IIPUSLIAA'] / df.loc[:'2005', 'GDPA']) / 10
data['Assets'] = (df.loc[:'2005', 'IIPUSASSA'] / df.loc[:'2005', 'GDPA']) / 10
data2 = pd.DataFrame()
data2['Liab'] = (df2.loc[:, 'IIPUSLIAQ'] / df2.loc[:, 'GDP']) / 10
data2['Assets'] = (df2.loc[:, 'IIPUSASSQ'] / df2.loc[:, 'GDP']) / 10
result = data.append(data2)
result.to_csv(data_dir / 'iip.csv', index_label='date')

lt = result.index[-1]
ltdt = dtxt(lt)['qtr1']
pr = '2006-01-01'
ltvala = result.loc[lt, 'Assets']
ltvall = result.loc[lt, 'Liab']
ltvaln = ltvala - ltvall
prvala = result.loc[pr, 'Assets']
prvall = result.loc[pr, 'Liab']
prvaln = prvala - prvall

text = (f'In {ltdt}, domestic holdings of foreign assets are equivalent '+
        f'to {ltvala:.1f} percent of GDP '+
        '(see {\color{blue!95!violet}\\textbf{---}}), compared to '+
        f'{prvala:.1f} percent in 2006 Q1. Domestic liabilities '+
        f'to the foreign sector total {ltvall:.1f} percent of GDP in {ltdt} '+
        '(see {\color{red}\\textbf{---}}), '+
        f'and {prvall:.1f} percent in 2006 Q1. '+
        'As a result, net IIP, holdings of foreign assets minus liabilities, identifies the '+
        'US as a net debtor to the result of the world, to the equivalent of '
        f'{abs(ltvaln):.1f} percent of GDP in {ltdt} '+
        '(see\cbox{cyan!25!white}) '+
        f'and {abs(prvaln):.1f} '+
        'percent of GDP in 2006 Q1.')

write_txt(text_dir / 'niip.txt', text)

print(text)

In 2020 Q1, domestic holdings of foreign assets are equivalent to 124.3 percent of GDP (see {\color{blue!95!violet}\textbf{---}}), compared to 106.3 percent in 2006 Q1. Domestic liabilities to the foreign sector total 180.2 percent of GDP in 2020 Q1 (see {\color{red}\textbf{---}}), and 118.5 percent in 2006 Q1. As a result, net IIP, holdings of foreign assets minus liabilities, identifies the US as a net debtor to the result of the world, to the equivalent of 56.0 percent of GDP in 2020 Q1 (see\cbox{cyan!25!white}) and 12.2 percent of GDP in 2006 Q1.
